In [6]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
#from fake_useragent import UserAgent
import requests
from selenium import webdriver
from collections import Counter
from tqdm.notebook import tqdm

import time
import openpyxl

from selenium import webdriver
import random

In [17]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [18]:
def url_list(q):
    wb = openpyxl.Workbook()
    sheet = wb.active
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
      "referer": "http://browse.auction.co.kr/"}
    row = "http://browse.auction.co.kr/search?keyword={}".format(q)
    main_,para = url_split(row)
    html = requests.get(main_,params=para,headers=headers).text
    soup = BeautifulSoup(html,"html.parser")
    reg = re.compile(r".+itemno=(.\d+)")
    al_url = []
    for tag in soup.select(".section--module_wrap div .itemcard div span"):
        try:
            url_candi = tag.find(class_="link--itemcard")["href"]
            m = reg.match(url_candi)
            al_url.append(m.group())

        except:
            pass
    #al_url = list(set(al_url))
    al_url_set = list(set(al_url))
    return al_url,al_url_set

In [19]:
def append_all(list_,da_):
    for i in range(len(list_)):
        list_[i].append(da_[i])

In [20]:
def update_list(review_list):
    ra2,id2,da2,con2,he2,cha2 = [],[],[],[],[],[]
    
    for i in range(len(review_list)):
        try:
            rate = review_list[i].find_element_by_css_selector('.box__star .sprite__vip[style]')
            ra = rate.get_attribute('style')
            rate_ = re.match(r".+: (\d+)%",ra).group(1).strip()
            id_ = review_list[i].find_element_by_css_selector('.box__info .text__writer').text
            date_ = review_list[i].find_element_by_css_selector('.box__info .text__date').text
            try:
                content_ = review_list[i].find_element_by_css_selector('.box__content .text').text
                help_ = review_list[i].find_element_by_css_selector('.box__helpful .text__count').text
            except:
                content_ = ""
                help_ = "0"
            cha_ = "Auction"
            ra2.append(rate_)
            id2.append(id_)
            da2.append(date_)
            con2.append(content_)
            he2.append(help_)
        except :
            pass
    return [ra2,id2,da2,con2,he2,cha2]

In [52]:
def auction_all(al_url_set,path_):
    for i in al_url_set:
        driver.get(i) 
        time.sleep(random.uniform(1,1.5))
        cnt = driver.find_elements_by_css_selector('ul .uxetabs_menu')
        cnt[1].click()
        time.sleep(random.uniform(1,1.5))
        #review_list = driver.find_elements_by_css_selector(".list__review li")
        product_ = driver.find_elements_by_css_selector("#frmMain > h1")[0].text.replace(" ","_").replace(".","").replace("/","_")
        print(product_)
        rate_list,id_list,date_list,content_list,help_list= [],[],[],[],[]
        List = [date_list,id_list,content_list,rate_list,help_list]
        try:
            total_p = int(driver.find_elements_by_css_selector('.box__page-jump .text__total em.text')[0].text)
            next_ = driver.find_elements_by_css_selector('.link__page-move.link__page-next')
            if len(next_) != 0:
                for m in tqdm(range(total_p)) :
                    review_list = driver.find_elements_by_css_selector(".list__review li")
                    li3 = update_list(review_list)
                    for k in range(len(li3)):
                        List[k].extend(li3[k])
                    next_ = driver.find_elements_by_css_selector('.link__page-move.link__page-next')
                    try:
                        next_[0].click()
                        time.sleep(random.uniform(1,1.5))
                    except:
                        break

            else:
                review_list = driver.find_elements_by_css_selector(".list__review li")
                li3 = update_list(review_list)
                for k in range(len(li3)):
                    List[k].extend(li3[k])
        except:
            pass
        df_ = pd.DataFrame(List).T.rename(columns={2:"date",1:"id",3:"contents",0:"score",4:"liked",})
        df_["channel"] = "Auction"
        df_.to_excel(path_.format(product_))

In [ ]:
#.rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})

# web driver

In [53]:
driver = webdriver.Chrome(r'C:\Users\Samsung\jupyter\crawling\chromedriver_win32\chromedriver')
#river2 = webdriver.PhantomJS(r"C:\Users\LAB\jupyter\zero2one\phantomjs-2.1.1-windows\phantomjs-2.1.1-windows\bin\phantomjs")

In [54]:
wb = openpyxl.Workbook()
sheet = wb.active

In [55]:
urls = url_list("헤어드라이기")[0]

In [56]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\auction\{}.xlsx"
auction_all(urls,path_)

한일전자_헤어드라이기_RH-4200_2200W_싸이클론


  0%|          | 0/57 [00:00<?, ?it/s]

로라루즈_헤어_드라이기_전자파차단_국제특허기술


  0%|          | 0/14 [00:00<?, ?it/s]

1700W_고출력_헤어드라이기_고풍량드라이기_CKI-D700


...

  0%|          | 0/5 [00:00<?, ?it/s]

PHILIPS_헤어드라이기_라인프렌즈_코니_BHC010_69


  0%|          | 0/3 [00:00<?, ?it/s]

(최종21110원)_듀얼_에어케어_헤어드라이기_BHD302_10
PHILIPS_듀얼_에어케어_헤어드라이기_BHD300_10


WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.164)
